In [19]:
%load_ext autoreload
%autoreload 2

In [54]:
import sys
sys.path.insert(0, '../')
from humannotator.data import Data_DataFrame
from humannotator.results import Question_MultipleChoice, Answer
from humannotator.interface import Stop, Interface
import pandas as pd

In [13]:
df = pd.read_csv('news.csv', index_col=0)

data = Data_DataFrame(df, elements='title', id='news_id')

In [9]:
data.elements

0                  Rand Paul wins 2015 CPAC straw poll
1    Can Singing Mice Reveal the Roots of Human Spe...
2    Dollar hits 15-month low against yen after Yel...
3    Trump's Supreme Court pick dispirited by presi...
4    FULL TEXT: 10 Things Milo Hates About Islam - ...
5    5 Border Horrors Establishment Media Mostly Ig...
6    Crew members injured as plane avoids near coll...
7    Bodies Of Missing Married Couple Found On Susp...
8    350 Square Feet, Two Kids, Two Cats and a Rabb...
9    CDC warns about deadly mushrooms amid surge in...
Name: title, dtype: object

In [10]:
data.ids

0    052632_2015-02-28
1    071607_2016-12-12
2    141694_2016-02-10
3    137157_2017-02-09
4    034187_2016-09-27
5    018678_2017-04-23
6    120386_2016-11-14
7    135236_2016-11-10
8    184514_2017-03-17
9    106098_2017-06-02
Name: news_id, dtype: object

In [64]:
q = Question_MultipleChoice(
    instruction="You're a poop!",
    choices={
        '0': 'not adverse media',
        '1': 'adverse media',
        '3': 'exclude from dataset'
        })
a = Answer(q)
i = Interface(a)

In [66]:
i()


You're a poop!
[0] - not adverse media
[1] - adverse media
[3] - exclude from dataset
[.] - exit



 .
